In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
from glob import glob
from time import time


device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
import sys
sys.path.append('../')

from Inference import Inference as inf
from models import ViT, configs
from utils import dataloader

In [4]:
data_loader_training, data_loader_validate, data_loader_test = dataloader.return_dataloaders(batch_size = 1, 
                                                                                            in_channels = 4, 
                                                                                            lds_ks = 10,
                                                                                            lds_sigma = 8, 
                                                                                            dw_alpha = 3.9, 
                                                                                            betha  = 4, 
                                                                                            reweighting_method = 'lds', 
                                                                                            resmapling_status = False,
                                                                                            exp_name = 't1')

(13875, 33) | (8233, 33) | (12443, 33)


In [17]:
sample = next(iter(data_loader_test))   
print(sample["image"].shape, sample["mask"].shape)
print(sample["EmbList"])

torch.Size([1, 4, 16, 16, 15]) torch.Size([1, 1, 16, 16, 1])
[tensor([2]), tensor([3]), tensor([11]), tensor([5])]


In [5]:
config = configs.SRTR_Configs(img_size = 16, patch_size = 8, embed_dim = 1024, mlp_dim = 512, 
                    in_channels = 4, out_channels = 15, num_heads = 8, num_layers = 6, cond = None,
                    Attn_drop = 0.1, Proj_drop = 0.1, PostNorm = False, vis = True, 
                    kernel_size = [3, 3], dilation = [1, 1], stride  = [1, 1]
                    ).call()

In [6]:
model = ViT.ours(config).to(device)
# best_model_name = '/data2/hkaman/DiT/EXPs/EXP_vanilla_mse_01_0001_1024_8_6/best_model_vanilla_mse_01_0001_1024_8_6.pth'
# model.load_state_dict(torch.load(best_model_name))

In [24]:
pred = model(sample["image"].to(device), sample["EmbList"])

r2, mae, rmse, mape, _,_ = inf.regression_metrics(sample["mask"][0, 0, :, :, 0].detach().cpu().numpy(), pred[0, 0, :, :].detach().cpu().numpy())
scores = (r'$R^2={:.2f}$' + '\n' + r'MAE={:.2f}' + '\n' + r'$RMSE={:.2f}$' + '\n' + r'$MAPE={:.2f}$').format(
    r2, mae, rmse, mape)
print(scores)

$R^2=-1.17$
MAE=1.23
$RMSE=1.71$
$MAPE=0.15$


In [ ]:
from models import engine
ImYiEst = engine.ImbYieldEst(model, 
                                lr = 0.01, 
                                wd = 0.0001, 
                                exp = 'vanilla_mse_01_0001_1024_8_6')

_ = ImYiEst.predict(model, data_loader_training, data_loader_validate, data_loader_test)

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim

# Example linear regression model
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.fc = nn.Linear(1, 1)

    def forward(self, x):
        return self.fc(x)

# Generate some random data for training
torch.manual_seed(42)
x = torch.rand((100, 1))
y = 2 * x + 1 + torch.randn((100, 1)) * 0.1  # Adding some noise

# Define model, loss function, and initial learning rates
model = LinearRegression()
criterion = nn.MSELoss()

# Learning rate for annealing (start high)
learning_rate_anneal = 0.01
optimizer_anneal = optim.SGD(model.parameters(), lr=learning_rate_anneal)

# Learning rate for warm-up (start low)
learning_rate_warmup = 0.001
optimizer_warmup = optim.SGD(model.parameters(), lr=learning_rate_warmup)

# Training loop with learning rate annealing
print("Training with Learning Rate Annealing...")
for epoch in range(200):
    optimizer_anneal.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer_anneal.step()
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/100], Loss: {loss.item():.4f}, Learning Rate: {learning_rate_anneal:.4f}")
    # Anneal the learning rate (e.g., reduce by a factor)
    learning_rate_anneal *= 0.95
    for param_group in optimizer_anneal.param_groups:
        param_group['lr'] = learning_rate_anneal



Training with Learning Rate Annealing...
Epoch [20/100], Loss: 2.7535, Learning Rate: 0.0038
Epoch [40/100], Loss: 2.1590, Learning Rate: 0.0014
Epoch [60/100], Loss: 1.9798, Learning Rate: 0.0005
Epoch [80/100], Loss: 1.9194, Learning Rate: 0.0002
Epoch [100/100], Loss: 1.8983, Learning Rate: 0.0001
Epoch [120/100], Loss: 1.8907, Learning Rate: 0.0000
Epoch [140/100], Loss: 1.8881, Learning Rate: 0.0000
Epoch [160/100], Loss: 1.8871, Learning Rate: 0.0000
Epoch [180/100], Loss: 1.8867, Learning Rate: 0.0000
Epoch [200/100], Loss: 1.8866, Learning Rate: 0.0000


In [17]:
# Training loop with learning rate warm-up
print("\nTraining with Learning Rate Warm-Up...")
for epoch in range(200):
    optimizer_warmup.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer_warmup.step()
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/100], Loss: {loss.item():.4f}, Learning Rate: {learning_rate_warmup:.4f}")
    # Warm-up the learning rate (e.g., increase linearly)
    learning_rate_warmup += 0.0001
    for param_group in optimizer_warmup.param_groups:
        param_group['lr'] = learning_rate_warmup

print("Training finished!")


Training with Learning Rate Warm-Up...
Epoch [20/100], Loss: 1.5768, Learning Rate: 0.0029
Epoch [40/100], Loss: 1.0794, Learning Rate: 0.0049
Epoch [60/100], Loss: 0.6160, Learning Rate: 0.0069
Epoch [80/100], Loss: 0.3048, Learning Rate: 0.0089
Epoch [100/100], Loss: 0.1448, Learning Rate: 0.0109
Epoch [120/100], Loss: 0.0793, Learning Rate: 0.0129
Epoch [140/100], Loss: 0.0564, Learning Rate: 0.0149
Epoch [160/100], Loss: 0.0480, Learning Rate: 0.0169
Epoch [180/100], Loss: 0.0434, Learning Rate: 0.0189
Epoch [200/100], Loss: 0.0395, Learning Rate: 0.0209
Training finished!
